# snapReactors

Copyright (c) Dan Kotlyar and CoRE group

# Materials Container

* This container stores materials information such as isotopic definition, abundances, uncertainties and material properties.
* The container also stores, modifies or creates new h5 files for data storage.

## Code: 

In [1]:
import numpy as np
import sympy as sp
from snapReactors.containers.materials import Material
from snapReactors.containers.property import Constant, Table, Correlation

## Defining a new material

### Define with Materials container

1. Give name of material
2. Give type of uncertainty, must be in ``Enum.UTYPE`` which are:
    - ABSOLUTE 
    - RELATIVE
    - PERCENTAGE
    - NONE
3. Give the composition type, must be in ``Enum.CTYPE`` which are:
    - WDENSITY (weight density g/cc)
    - ADENSITY (atomic density a/b-cm)
    - WEIGHT (weight fraction)
    - ATOMIC (atomic fraction)
4. Give the isotopes that define a material as a np.array
5. Give the abundances for each isotope as a np.array
6. The uncertainty value, reference, description, and properties are left as optional parameters.
    1. Note that properties must be under ``ALLOWED_PROPERTIES``


In [8]:
mat1 = Material("mat1", 'NONE', 'WEIGHT', np.array([]), np.array([]), None, reference=None, description='This is an example', _properties=None)
print(mat1)

{'id': 'mat1', 'utype': <UTYPE.NONE: 4>, 'ctype': <CTYPE.WEIGHT: 2>, 'abundances': array([], dtype=float64), 'isotopes': array([], dtype=float64), 'unc': None, 'reference': None, 'description': 'This is an example', '_properties': [], '_propertiesDict': {}, 'referenceCalcFile': None, 'isVerified': None, 'dependencyDict': {}}


#### Properties may be inputted as a list of properties or a single property

In [11]:
p1 = Constant(id='cv',  value=1, unit= "J/kg/K", unc=None, ref=None, description=None)
mat2 = Material("mat2", 'NONE', 'WEIGHT', np.array([]), np.array([]), None, reference=None, description='This is an example', _properties=[p1])
print(mat2)

p2 = Table('h',  np.array([1, 2, 3, 4]), 'W/K/m^2', np.array([100, 200, 300, 400]), 'K',  unc = np.array([.01, .01, .01, .01]), dependency2=None, dependencyUnit2=None, ref=None, description=None)
corr1 = "T**2 + P + 1/2"
syms1 = "T, P"
p3 = Correlation('h', corr1, syms1, 'W/K/m^2', np.array([300, 600]), 'K', np.array([10, 20]), 'Pa', unc=None, ref=None, description=None)
properties = [p2, p3]
mat3 = Material("mat3", 'NONE', 'WEIGHT', np.array([]), np.array([]), None, reference=None, description='This is an example', _properties=properties)
print(mat3)

{'id': 'mat2', 'utype': <UTYPE.NONE: 4>, 'ctype': <CTYPE.WEIGHT: 2>, 'abundances': array([], dtype=float64), 'isotopes': array([], dtype=float64), 'unc': None, 'reference': None, 'description': 'This is an example', '_properties': [<snapReactors.containers.property.Constant object at 0x0000023E2913C2E0>], '_propertiesDict': {'cv': <snapReactors.containers.property.Constant object at 0x0000023E2913C2E0>}, 'referenceCalcFile': None, 'isVerified': None, 'dependencyDict': {}}
{'id': 'mat3', 'utype': <UTYPE.NONE: 4>, 'ctype': <CTYPE.WEIGHT: 2>, 'abundances': array([], dtype=float64), 'isotopes': array([], dtype=float64), 'unc': None, 'reference': None, 'description': 'This is an example', '_properties': [<snapReactors.containers.property.Table object at 0x0000023E2913C0D0>, <snapReactors.containers.property.Correlation object at 0x0000023E2CB3C7C0>], '_propertiesDict': {'h': <snapReactors.containers.property.Correlation object at 0x0000023E2CB3C7C0>}, 'referenceCalcFile': None, 'isVerified'

### Reading in isotopic defintion through a text file

The ``Materials._materialReader(materialData)`` method is used to save materials data that was read in from a text file. There are several rules to keep in mind for the structure of the text file:
1. The ``Material Name`` must be indicated at the beginning of every material data section with the following format:
    - Material Name: Example Name
2. The ``ctype``, ``utype``, and ``Number of isotopes`` must be indicated before the beginning of ``Isotopic Definition`` although their order doesn't matter. 
3. The ``Isotopic Definition`` must have a line between itself and where the definition begins. In the example below a dashed line is used to indicate this seperation.
4. For each input a colon is used to seperate the keyword and input, for example:
    - utype: NONE
5. The location of ``Reference`` and ``Description`` for a specific material must be placed before the beginning of the next ``Material Name`` if present. 

Material Property data can be read in by adding a ``Properties`` section. 
1. The location of ``Properties`` must be before the beginning of the next ``Material Name`` and is indicated with curly brackets:
   ```
    Properties: {
        
    }
    ```
2. The formatting for the ``Properties`` information only requires there to be a colon in between the keyword and value, and that each keyword be on its own line
    ```
        type = const, table, corr
        id = property id
        unit = SI or imperial
        must have a ":" between keyword and value i.e "keyword: value"
        each keyword must on its own line i.e 
         keyword1: val1 
         keyword: val2

        array type inputs are denoted using "[]" i.e [1, 2] or [1 2] 
        multidimensional arrays can be denoted using the ";" matlab style i.e
         [1 2; 3 4] or [1, 2;
                        3, 4]
         or by using a newline i.e
           [1 2
            3 4] 
        Supports comments by preceeding a line with "%"
        Examples are included below
    }
    ```
3. Structure of ``Properties`` input is outlined in Property Container documentation.

Optional parameters such as reference or uncertainty values can be left out, however, warnings will be highlighted to the user. Two examples for the Material Property data are shown below.

####  Example text file shown below

In [20]:
text_file = open('C:\\Users\\sgarc\\OneDrive\\Documents\\SNAP-REACTORS\\snapReactors\\jupyter_notebooks\\test.txt')
file_content = text_file.read()
print(file_content)
text_file.close()

Material Name: fuel
ctype: ADENSITY
utype: NONE
Number of isotopes: 9
Isotopic Definition:
-------------------
 1001   5.960E-2
 1002   8.790E-6
92235   1.430E-3
92238   1.040E-4
40090   1.830E-2
40091   4.000E-3
40092   6.110E-3
40094   6.190E-3
40096   9.980E-4

Properties: {
%property values for material
%type = const, table, corr
%id = property id
%unit = SI or imperial
%must have a ":" between keyword and value i.e "keyword: value"
%each keyword must on its own line i.e 
% keyword1: val1 
% keyword: val2
%array type inputs are denoted using "[]" i.e [1, 2] or [1 2] 
%multidimensional arrays can be denoted using the ";" matlab style i.e
% [1 2; 3 4] or [1, 2;
%                3, 4]
% or by using a newline i.e
%   [1 2
     3 4] 
%Supports comments by preceeding a line with "%"
%Examples are included below

type:const
id:r
unit:SI 
value:[6060]
unc:[.01]

type:table 
id:h 
unit:imperial 
ref:NAA-SR-6160 
dep1unit:K 
dep1values: [1 2]
dep2unit:Pa 
dep2values: [.1 .2]
value: [1.1 2.1


#### Materials definition returned by readData

In [22]:
text_file = open('C:\\Users\\sgarc\\OneDrive\\Documents\\SNAP-REACTORS\\snapReactors\\jupyter_notebooks\\test.txt')
materialData = text_file.readlines()
mats = Material._materialReader(materialData)
print(mats)
text_file.close()

[<snapReactors.containers.materials.Material object at 0x0000023E2CCB8670>, <snapReactors.containers.materials.Material object at 0x0000023E2CCB8070>, <snapReactors.containers.materials.Material object at 0x0000023E2CB3C670>, <snapReactors.containers.materials.Material object at 0x0000023E2CB3C0D0>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA64C0>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA6790>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA6F40>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA63D0>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA6820>, <snapReactors.containers.materials.Material object at 0x0000023E2CCA62E0>]


C:\Users\sgarc\AppData\Roaming\Python\Python39\site-packages\snapreactors-0.0.1-py3.9.egg\snapReactors\containers\property.py:683: InputFileSyntaxWarning: reference not given for r const property @ line: 19
  warnings.warn("reference not given for {} {} property @"
C:\Users\sgarc\AppData\Roaming\Python\Python39\site-packages\snapreactors-0.0.1-py3.9.egg\snapReactors\containers\property.py:683: InputFileSyntaxWarning: reference not given for r const property @ line: 2
  warnings.warn("reference not given for {} {} property @"


## Updating properties to materials

1. The properties must be from the following list: ['cp', 'cv', 'g', 'h', 'my', 'pr', 'r', 's', 'tc', 'v']

In [23]:
p4 = Constant(id='cv',  value=1, unit= "J/kg/K", unc=None, ref=None, description=None)
print(p4)

mat3.addproperty([p4])
print(mat3)


{'id': 'cv', 'dtype': <DTYPE.NUMBER: 1>, 'vtype': <VTYPE.CONSTANT: 1>, 'value': array([1]), 'valueUnit': 'J/kg/K', 'unc': None, 'dependents': None, 'dependentsUnit': None, 'description': None, 'ref': None}
{'id': 'mat3', 'utype': <UTYPE.NONE: 4>, 'ctype': <CTYPE.WEIGHT: 2>, 'abundances': array([], dtype=float64), 'isotopes': array([], dtype=float64), 'unc': None, 'reference': None, 'description': 'This is an example', '_properties': [<snapReactors.containers.property.Table object at 0x0000023E2913C0D0>, <snapReactors.containers.property.Correlation object at 0x0000023E2CB3C7C0>, <snapReactors.containers.property.Constant object at 0x0000023E2CCA6370>], '_propertiesDict': {'h': <snapReactors.containers.property.Correlation object at 0x0000023E2CB3C7C0>, 'cv': <snapReactors.containers.property.Constant object at 0x0000023E2CCA6370>}, 'referenceCalcFile': None, 'isVerified': None, 'dependencyDict': {}}
